In [1]:
 pip install -Uq langgraph langsmith langchain_groq langchain_tavily grandalf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.2/386.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.0 MB/s eta 0:00:00


In [3]:
from os import environ
from google.colab import userdata

environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

environ["LANGCHAIN_TRACING_V2"] = "true"
environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
environ["LANGCHAIN_PROJECT"] = "03-LangGraph-Agent"

In [4]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [6]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=3)
tools = [search]

In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-20b")
llm_with_tools = llm.bind_tools(tools)

In [8]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
  messages: Annotated[list, add_messages]

In [9]:
def chatbot(state: State) -> State:
  answer = llm_with_tools.invoke(state["messages"])
  return State(messages=[answer])

In [17]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(State)
graph_builder.add_node("llm", chatbot)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_conditional_edges("llm", tools_condition, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "llm")
graph_builder.set_entry_point("llm")
graph = graph_builder.compile(checkpointer=memory)

In [18]:
print(graph.get_graph().draw_ascii())

        +-----------+       
        | __start__ |       
        +-----------+       
              *             
              *             
              *             
          +-----+           
          | llm |           
          +-----+.          
          .       .         
        ..         ..       
       .             .      
+---------+       +-------+ 
| __end__ |       | tools | 
+---------+       +-------+ 


In [21]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "1"})

inputs = {"messages":[("user", "내 이름은 철수 입니다. 만나서 반가워요")]}

for event in graph.stream(inputs, config):
  for value in event.values():
    print(value["messages"][-1].content)

안녕하세요, 철수님! 다시 만나서 반갑습니다. 무엇을 도와드릴까요?


In [22]:
inputs = {"messages":[("user", "제 이름을 기억하나요?")]}

for event in graph.stream(inputs, config):
  for value in event.values():
    print(value["messages"][-1].content)

네, 기억하고 있어요. 제 이름은 철수라고 하셨죠? 😄 어떤 도움이 필요하신가요?


In [27]:
# 스냅샷 확인하기
config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='내 이름은 철수 입니다. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='80044f54-4eb6-41ac-9b33-e8eb55502058'), AIMessage(content='안녕하세요, 철수님! 만나서 반갑습니다. 오늘은 어떻게 도와드릴까요?', additional_kwargs={'reasoning_content': 'The user says in Korean: "내 이름은 철수 입니다. 만나서 반가워요" meaning "My name is Cheolsu. Nice to meet you." The assistant should respond in Korean, greeting back, acknowledging name. No tool needed.'}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 1369, 'total_tokens': 1453, 'completion_time': 0.074356909, 'prompt_time': 0.089242463, 'queue_time': 0.002878982, 'total_time': 0.163599372}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_5d01058a94', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--829769de-0f59-4140-af4e-204d02f1ffb3-0', usage_metadata={'input_tokens': 1369, 'output_tokens': 84, 'total_tokens': 1453}), HumanMessage(content='내 이름은 철수 입니다. 

In [28]:
snapshot.values["messages"]

[HumanMessage(content='내 이름은 철수 입니다. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='80044f54-4eb6-41ac-9b33-e8eb55502058'),
 AIMessage(content='안녕하세요, 철수님! 만나서 반갑습니다. 오늘은 어떻게 도와드릴까요?', additional_kwargs={'reasoning_content': 'The user says in Korean: "내 이름은 철수 입니다. 만나서 반가워요" meaning "My name is Cheolsu. Nice to meet you." The assistant should respond in Korean, greeting back, acknowledging name. No tool needed.'}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 1369, 'total_tokens': 1453, 'completion_time': 0.074356909, 'prompt_time': 0.089242463, 'queue_time': 0.002878982, 'total_time': 0.163599372}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_5d01058a94', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--829769de-0f59-4140-af4e-204d02f1ffb3-0', usage_metadata={'input_tokens': 1369, 'output_tokens': 84, 'total_tokens': 1453}),
 HumanMessage(content='내 이름은 철수 입니다. 만나서 반가워요', additional_kwargs={},

In [29]:
snapshot.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f095f90-dc31-60d3-800a-e7ce1dfdd00b'}}

In [30]:
# 다음 노드 출력
snapshot.next

()